In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py


In [2]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc

pd.options.display.expand_frame_repr = False
pd.set_option("display.max_columns", 20)

warnings.simplefilter('ignore')
from sklearn.metrics import roc_auc_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, learning_curve
from sklearn.utils import shuffle

import riiideducation

types = {
    'row_id': 'int64',
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'content_type_id': 'int8',
    'task_container_id': 'int16',
    'user_answer': 'int8',
    'answered_correctly': 'int64',
    'prior_question_elapsed_time': 'float32',
    'prior_question_had_explanation': 'boolean'
}

train1 = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=2.5 * (10 ** 6),
    skiprows=0,
    dtype=types
)

train2 = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=2.5 * (10 ** 6),
    skiprows=2 * (10 ** 7),
    dtype=types
)

train3 = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=2.5 * (10 ** 6),
    skiprows=4 * (10 ** 7),
    dtype=types
)

train4 = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=2.5 * (10 ** 6),
    skiprows=6 * (10 ** 7),
    dtype=types
)

train5 = pd.read_csv(
    '/kaggle/input/riiid-test-answer-prediction/train.csv',
    low_memory=False,
    nrows=2.5 * (10 ** 6),
    skiprows=8 * (10 ** 7),
    dtype=types
)

train2.columns = train1.columns
train3.columns = train1.columns
train4.columns = train1.columns
train5.columns = train1.columns

train = pd.concat([train1, train2, train3, train4, train5], axis=0, ignore_index=True)

# --------------------------------FEATURE 4------------------------------------------------------
grouped_by_container_df = train.groupby('task_container_id')
container_df = grouped_by_container_df.agg(
    {'answered_correctly': [
        'mean',
        'count'],
        'content_type_id': ['sum']
    }).copy()

container_df.columns = [
    'mean_container_acc',
    'container_unit_count',
    'lecs_in_container']
container_df["lecs_in_container"] = container_df["lecs_in_container"].astype(int)
container_df["lecs_to_qs"] = container_df["lecs_in_container"] / container_df["container_unit_count"]

container_df
del [[train]]
gc.collect()
# --------------------------------FEATURE 4------------------------------------------------------

# shift timstamp for 11 feature
train_questions_only_df1 = train1[train1['answered_correctly'] != -1].copy()
train_questions_only_df2 = train2[train2['answered_correctly'] != -1].copy()
train_questions_only_df3 = train3[train3['answered_correctly'] != -1].copy()
train_questions_only_df4 = train4[train4['answered_correctly'] != -1].copy()
train_questions_only_df5 = train5[train5['answered_correctly'] != -1].copy()

train_questions_only_df1['question_elapsed_time'] = train_questions_only_df1['prior_question_elapsed_time'].shift(-1)
train_questions_only_df1['question_had_explanation'] = train_questions_only_df1['prior_question_had_explanation'].shift(-1)
train_questions_only_df1['ts_shift'] = train_questions_only_df1['timestamp'].shift(+1)

train_questions_only_df2['question_elapsed_time'] = train_questions_only_df2['prior_question_elapsed_time'].shift(-1)
train_questions_only_df2['question_had_explanation'] = train_questions_only_df2['prior_question_had_explanation'].shift(-1)
train_questions_only_df2['ts_shift'] = train_questions_only_df2['timestamp'].shift(+1)

train_questions_only_df3['question_elapsed_time'] = train_questions_only_df3['prior_question_elapsed_time'].shift(-1)
train_questions_only_df3['question_had_explanation'] = train_questions_only_df3['prior_question_had_explanation'].shift(-1)
train_questions_only_df3['ts_shift'] = train_questions_only_df3['timestamp'].shift(+1)

train_questions_only_df4['question_elapsed_time'] = train_questions_only_df4['prior_question_elapsed_time'].shift(-1)
train_questions_only_df4['question_had_explanation'] = train_questions_only_df4['prior_question_had_explanation'].shift(-1)
train_questions_only_df4['ts_shift'] = train_questions_only_df4['timestamp'].shift(+1)

train_questions_only_df5['question_elapsed_time'] = train_questions_only_df5['prior_question_elapsed_time'].shift(-1)
train_questions_only_df5['question_had_explanation'] = train_questions_only_df5['prior_question_had_explanation'].shift(-1)
train_questions_only_df5['ts_shift'] = train_questions_only_df5['timestamp'].shift(+1)

train_questions_only_df = pd.concat([train_questions_only_df1, train_questions_only_df2, train_questions_only_df3,
                                     train_questions_only_df4, train_questions_only_df5], axis=0, ignore_index=True)
train_questions_only_df = train_questions_only_df.drop(columns=['prior_question_elapsed_time',
                                                                'prior_question_had_explanation'])

del [[train_questions_only_df1]], [[train_questions_only_df2]], [[train_questions_only_df3]], \
    [[train_questions_only_df4]], [[train_questions_only_df5]]
gc.collect()


train_lecs_only_df1 = train1.copy()
train_lecs_only_df2 = train2.copy()
train_lecs_only_df3 = train3.copy()
train_lecs_only_df4 = train4.copy()
train_lecs_only_df5 = train5.copy()

train_lecs_only_df1['ts_shift'] = train_lecs_only_df1['timestamp'].shift(+1)
train_lecs_only_df2['ts_shift'] = train_lecs_only_df2['timestamp'].shift(+1)
train_lecs_only_df3['ts_shift'] = train_lecs_only_df3['timestamp'].shift(+1)
train_lecs_only_df4['ts_shift'] = train_lecs_only_df4['timestamp'].shift(+1)
train_lecs_only_df5['ts_shift'] = train_lecs_only_df5['timestamp'].shift(+1)

train_lecs_only_df1 = train_lecs_only_df1[train_lecs_only_df1['content_type_id'] == 1].copy()
train_lecs_only_df2 = train_lecs_only_df2[train_lecs_only_df2['content_type_id'] == 1].copy()
train_lecs_only_df3 = train_lecs_only_df3[train_lecs_only_df3['content_type_id'] == 1].copy()
train_lecs_only_df4 = train_lecs_only_df4[train_lecs_only_df4['content_type_id'] == 1].copy()
train_lecs_only_df5 = train_lecs_only_df5[train_lecs_only_df5['content_type_id'] == 1].copy()

train_lecs_only_df = pd.concat([train_lecs_only_df1, train_lecs_only_df2, train_lecs_only_df3,
                                     train_lecs_only_df4, train_lecs_only_df5], axis=0, ignore_index=True)
train_lecs_only_df = train_lecs_only_df.drop(columns=['prior_question_elapsed_time', 'prior_question_had_explanation'])

del [[train_lecs_only_df1]], [[train_lecs_only_df2]], [[train_lecs_only_df3]], \
    [[train_lecs_only_df4]], [[train_lecs_only_df5]]
gc.collect()

del [[train1]], [[train2]], [[train3]], \
    [[train4]], [[train5]]
gc.collect()

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                        dtype={'question_id': 'int16', 'part': 'int8', 'bundle_id': 'int16', 'tags': 'str'})
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv',
                       dtype={'lectures_id': 'int16', 'tag': 'int16', 'part': 'int8', 'type_of': 'str'})
ex_test = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

# --------------------------------FEATURE1------------------------------------------------------
grouped_by_user_df = train_questions_only_df.groupby('user_id')

user_ans_df = grouped_by_user_df.agg(
    {
        'answered_correctly': [
            'mean',
            'count']}
).copy()

user_ans_df.columns = [
    'mean_user_accuracy',
    'questions_answered'
]

user_ans_df
# --------------------------------FEATURE1------------------------------------------------------

# --------------------------------FEATURE 11.1------------------------------------------------------
train_questions_only_df.loc[train_questions_only_df['timestamp'] == 0, 'timestamp'] = 0
train_questions_only_df.loc[train_questions_only_df['timestamp'] == 0, 'ts_shift'] = 0
train_questions_only_df['spent_time'] = train_questions_only_df['timestamp'] - train_questions_only_df['ts_shift'] \
                                        - train_questions_only_df['question_elapsed_time']
train_questions_only_df.loc[train_questions_only_df['spent_time'] <= 0, 'spent_time'] = train_questions_only_df['spent_time'].mean()

spend_user_qu_df = train_questions_only_df.groupby('user_id')
spend_user_qu_df = spend_user_qu_df.agg({'spent_time': ['mean']}).copy()

spend_user_qu_df.columns = ['mean_spend_user_qu']

spend_user_qu_df[spend_user_qu_df['mean_spend_user_qu'] < 0]
train_questions_only_df[train_questions_only_df['spent_time'] < 0]

train_questions_only_df = train_questions_only_df.drop(columns=['spent_time', 'ts_shift', 'timestamp'])
# --------------------------------FEATURE11.1------------------------------------------------------

# --------------------------------FEATURE 11.2------------------------------------------------------
train_lecs_only_df.loc[train_lecs_only_df['timestamp'] == 0, 'timestamp'] = 0
train_lecs_only_df.loc[train_lecs_only_df['timestamp'] == 0, 'ts_shift'] = 0
train_lecs_only_df['spent_time'] = train_lecs_only_df['timestamp'] - train_lecs_only_df['ts_shift']
train_lecs_only_df.loc[train_lecs_only_df['timestamp'] == 0, 'spent_time'] = train_lecs_only_df['spent_time'].mean()

spend_user_lec_df = train_lecs_only_df.groupby('user_id')
spend_user_lec_df = spend_user_lec_df.agg({'spent_time': ['mean']}).copy()

spend_user_lec_df.columns = ['mean_spend_user_lec']
train_lecs_only_df[train_lecs_only_df['spent_time'] < 0]


spend_user_lec_df

del [[train_lecs_only_df]]
gc.collect()

# --------------------------------FEATURE11.2------------------------------------------------------

# --------------------------------FEATURE5-6------------------------------------------------------
# пока что просто среднее время на ответ без разгроничения на правильность
user_ans_time_df = grouped_by_user_df.agg(
    {
        'question_elapsed_time': [
            'mean']}
).copy()

user_ans_time_df.columns = ['mean_user_ans_time']

user_ans_time_df

del [[grouped_by_user_df]]
gc.collect()
# --------------------------------FEATURE5-6------------------------------------------------------

# --------------------------------FEATURE 2------------------------------------------------------
grouped_by_content_df = train_questions_only_df.groupby('content_id')
content_answers_df = grouped_by_content_df.agg(
    {
        'answered_correctly': [
            'mean',
            'count'
        ]
    }
).copy()

content_answers_df.columns = [
    'mean_que_accuracy',
    'question_was_asked'
]

content_answers_df
# --------------------------------FEATURE 2------------------------------------------------------


# --------------------------------FEATURE ?------------------------------------------------------
que_ans_time_df = grouped_by_content_df.agg(
    {
        'question_elapsed_time': [
            'mean']}
).copy()

que_ans_time_df.columns = [
    'mean_que_ans_time'
]

que_ans_time_df

del [[grouped_by_content_df]]
gc.collect()
# --------------------------------FEATURE ?------------------------------------------------------

train_questions_only_well = train_questions_only_df[train_questions_only_df['answered_correctly'] != 0].copy()

# --------------------------------FEATURE 7-8------------------------------------------------------
grouped_by_questions_well = train_questions_only_well.groupby('content_id')
quest_well_df = grouped_by_questions_well.agg(
    {
        'question_elapsed_time': ['mean']
    }).copy()
quest_well_df.columns = ['mean_que_ans_time_well']

quest_well_df["mean_que_ans_time_well"] = quest_well_df["mean_que_ans_time_well"].astype(int)

quest_well_df

del [[grouped_by_questions_well]]
gc.collect()
# --------------------------------FEATURE 7-8------------------------------------------------------

# --------------------------------FEATURE 5-6------------------------------------------------------

grouped_by_user_well = train_questions_only_well.groupby('user_id')
user_well_df = grouped_by_user_well.agg(
    {
        'question_elapsed_time': ['mean']
    }).copy()
user_well_df.columns = ['mean_user_ans_time_well']
user_well_df.fillna(max)
# user_well_df["mean_prior_question_elapsed_time_user"] = user_well_df["mean_prior_question_elapsed_time_user"].astype(int)

user_well_df

del [[train_questions_only_well]]
gc.collect()

del [[grouped_by_user_well ]]
gc.collect()
# --------------------------------FEATURE 5-6------------------------------------------------------


questions = questions.dropna()
questions
import re
import sys

# --------------------------------tags unitar------------------------------------------------------
s = set()
# dirt_list = questions["tags"]
dirt_list = questions["tags"].str.split(" ", expand=True)
for merge in dirt_list:
    ns = set(dirt_list[merge])
    s.update(ns)

s

s = {i for i in s if pd.notna(i)}

tag_df = pd.DataFrame(index=questions.value_counts(), columns=s)
tag_df = tag_df.fillna(0)
# tag_df.index = [i for i in range(13523)]
tag_df

buf = []
tagg_df = pd.DataFrame(index=questions.value_counts())
tagg_df.index = [i for i in range(len(questions.value_counts()))]
# tagg_df.index = questions.index

f = questions['tags']
for i in s:
    buf = []
    for j in range(len(f.values)):
        d = f.values[j].split(' ')
        # print(d)
        if i in d:
            buf.append(1)
        else:
            buf.append(0)
    df = pd.DataFrame(buf, columns=[i])
    # print(df)
    tagg_df = tagg_df.join(df)
tagg_df

# не трогать
questions.index = [i for i in range(len(questions.value_counts()))]
questions
# --------------------------------tags unitar------------------------------------------------------

# --------------------------------klusters tags------------------------------------------------------
# todo: {  [TAGS feature]
from gensim import corpora, models, matutils
from sklearn.cluster import KMeans

'''X = tagg_df
distorsions = []
# %%
for k in range(2, 150):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X)
    distorsions.append(kmeans.inertia_)
# %%
fig = plt.figure(figsize=(10, 10))
plt.plot(range(2, 150), distorsions)
plt.grid(True)
plt.title('Elbow curve')
plt.show()'''
# %%
# оптимальное число кластеров = 40

kmeans = KMeans(n_clusters=40)
km = kmeans.fit_predict(tagg_df)
print(km)

# %%

tagg_df['tag_clust'] = km
tagg_df

# todo:  [TAGS feature] }
# --------------------------------klusters tags------------------------------------------------------

# --------------------------------adding tags to questions------------------------------------------------------
tagg_df['index'] = tagg_df.index
questions['index'] = questions.index
questions = questions.merge(tagg_df, how='left', on='index')

questions = questions.drop(columns=[i for i in list(questions.columns) if (i != 'question_id') & (i != 'tag_clust')])
questions.rename(columns=lambda x: x.replace('question_id', 'content_id'), inplace=True)
questions
# --------------------------------adding tags to questions------------------------------------------------------

# --------------------------------MERGE TRAIN------------------------------------------------------
target = 'answered_correctly'
# merge all features in one df
train_questions_only_df = train_questions_only_df.merge(user_ans_df, how='left', on='user_id')

train_questions_only_df = train_questions_only_df.merge(content_answers_df, how='left', on='content_id')

train_questions_only_df = train_questions_only_df.merge(container_df, how='left', on='task_container_id')

train_questions_only_df = train_questions_only_df.merge(user_ans_time_df, how='left', on='user_id')

train_questions_only_df = train_questions_only_df.merge(que_ans_time_df, how='left', on='content_id')

train_questions_only_df = train_questions_only_df.merge(quest_well_df, how='left', on='content_id')

train_questions_only_df = train_questions_only_df.merge(user_well_df, how='left', on='user_id')

train_questions_only_df = train_questions_only_df.merge(questions, how='left', on='content_id')

train_questions_only_df = train_questions_only_df.merge(spend_user_lec_df, how='left', on='user_id')

train_questions_only_df = train_questions_only_df.merge(spend_user_qu_df, how='left', on='user_id')

train_questions_only_df
# --------------------------------MERGE TRAIN------------------------------------------------------

# --------------------------------CLEANING------------------------------------------------------
train_questions_only_df['question_had_explanation'] = train_questions_only_df['question_had_explanation'].fillna(value=False).astype(bool)
train_questions_only_df = train_questions_only_df.fillna(train_questions_only_df.mean())

col_to_drop = ['row_id', 'content_type_id', 'task_container_id', 'user_answer']
train_questions_only_df = train_questions_only_df.drop(columns=col_to_drop)

train_questions_only_df
gc.collect()
# --------------------------------CLEANING------------------------------------------------------

# --------------------------------PREPARE FOR TRAIN------------------------------------------------------
features = ['user_id', 'content_id', 'question_elapsed_time',
            'question_had_explanation', 'mean_user_accuracy', 'questions_answered',
            'mean_que_accuracy', 'question_was_asked', 'mean_container_acc',
            'container_unit_count', 'lecs_in_container', 'lecs_to_qs',
            'mean_user_ans_time', 'mean_que_ans_time', 'mean_que_ans_time_well',
            'mean_user_ans_time_well', 'tag_clust', 'mean_spend_user_lec', 'mean_spend_user_qu']

features

train_questions_only_df['question_had_explanation'] = train_questions_only_df['question_had_explanation'].astype(bool)

train_df = train_questions_only_df[features].copy()
y_train = train_questions_only_df[target].copy()

del [[train_questions_only_df]]
gc.collect()

tr_mean = train_df.mean()

# --------------------------------PREPARE FOR TRAIN------------------------------------------------------

# --------------------------------XGBOOST------------------------------------------------------
import xgboost
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

XGBmodel = xgboost.XGBClassifier(eta=0.3, max_depth=11, alpha=0.03, n_estimators=175, nthreads=-1)

XGBmodel.fit(train_df, y_train)
# %%
print(XGBmodel)

del [[train_df]]
del [[y_train]]
gc.collect()
# --------------------------------XGBOOST------------------------------------------------------

[19 34 34 ...  4  0  5]
[16:00:26] WARNING: ../src/learner.cc:516: 
Parameters: { nthreads } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:00:27] WARNING: ../src/gbm/gbtree.cc:139: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
XGBClassifier(alpha=0.03, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, eta=0.3, gamma=0,
              gpu_id=-1, importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=175, n_jobs=0, nthreads=-1, num_par

54

In [3]:
## SUBMISSION
env = riiideducation.make_env()
iter_test = env.iter_test()

In [4]:
for (test_df, sample_prediction_df) in iter_test:
    # merge
    test_df['question_elapsed_time'] = test_df['prior_question_elapsed_time'].shift(-1)
    test_df['question_had_explanation'] = test_df['prior_question_had_explanation'].shift(-1)
        
    test_df = test_df.merge(user_ans_df, how='left', on='user_id')
    
    test_df = test_df.merge(content_answers_df, how='left', on='content_id')
    
    test_df = test_df.merge(container_df, how='left', on='task_container_id')
    
    test_df = test_df.merge(user_ans_time_df, how='left', on='user_id')
    
    test_df = test_df.merge(que_ans_time_df, how='left', on='content_id')
    
    test_df = test_df.merge(quest_well_df, how='left', on='content_id')
    
    test_df = test_df.merge(user_well_df, how='left', on='user_id')
    
    test_df = test_df.merge(questions, how='left', on='content_id')
    
    test_df = test_df.merge(spend_user_lec_df, how='left', on='user_id')
    
    test_df = test_df.merge(spend_user_qu_df, how='left', on='user_id')
        
    test_df['question_had_explanation'] = test_df['question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(tr_mean, inplace = True)
        
    
    # preds
    test_df['answered_correctly'] = XGBmodel.predict_proba(test_df[features])[:, 1]
    cols_to_submission = ['row_id', 'answered_correctly', 'group_num']
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])